In [1]:
## importing packages
import torch
import os
import sys
import joypy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

## exposing path 
sys.path.insert(0, "../")

from int_filt.experiments import create_experiment
from int_filt.utils.config import configuration
from int_filt.utils.utils import ensure_reproducibility

In [2]:
## globals
ACTIVATIONS = {
    "relu": torch.nn.ReLU()
}

OPTIMIZERS = {
    "adam": torch.optim.Adam,
    "adam-w": torch.optim.AdamW
}

SCHEDULERS = {
    "none": None
}

DEVICES = {
    "cpu": torch.device("cpu"),
    "cuda": torch.device("cuda")
}

## defining settings
num_dims = 1
#b_net_amortized = True
b_net_amortized = False
sigma_x = 1e-2
sigma_y = 1e-2

## plotting
num_observations_to_plot = 1000
num_iters_to_plot = 200

Exponential non linearity

In [3]:
## defining configurations
experiment_conf = {
    "experiment": "nlg",
    "non_linearity": "exp",
    "num_dims": num_dims,
    "sigma_x": sigma_x,
    "sigma_y": sigma_y,
    "b_net_amortized": b_net_amortized,
    "b_net_lr": 1e-3,
    "b_net_num_grad_steps": 2000
}
## parsing default arguments
args = configuration(args=[])
## retrieving activations
args.b_net_activation = ACTIVATIONS[args.b_net_activation]
## retrieving device
args.device = DEVICES[args.device]
## creating experiment
args = vars(args)

## setting current configurations
for k, v in experiment_conf.items():
    args[k] = v

## adding mc configuration
args["mc_config"] = {"num_samples": args["num_samples"]}

## prepare for training drift
b_net_num_grad_step = args["b_net_num_grad_steps"]
b_net_optimizer = args["b_net_optimizer"]
b_net_scheduler = args["b_net_scheduler"]
b_net_lr = args["b_net_lr"]

## dump dir 
dump_dir = args["dump_dir"]
path = Path(dump_dir)
path.mkdir(parents=True, exist_ok=True)

## reproducibility
random_seed = args["random_seed"]
ensure_reproducibility(random_seed)

## displaying current arguments
print(args)

## creating experiment
experiment_nlg_exp = create_experiment(args)

## joyplot
## retrieving data
latent_states_nlg_exp = torch.squeeze(experiment_nlg_exp.ssm.sim["latent_states"]).numpy().T
observations_nlg_exp = torch.squeeze(experiment_nlg_exp.ssm.sim["latent_states"]).numpy().T
## constructing data frame
#observation_indices = np.arange(args["num_iters"])
observation_indices = np.arange(num_iters_to_plot)
latent_states_nlg_exp = pd.DataFrame(latent_states_nlg_exp[:num_observations_to_plot, :num_iters_to_plot], columns = observation_indices)
## ridge plot
#fig, axes = joypy.joyplot(latent_states_nlg_exp, ylabels=False)

{'interpolant_method': 'pffp_v0', 'num_samples': 100, 'backbone': 'mlp', 'b_net_hidden_dims': [16], 'b_net_activation': ReLU(), 'b_net_activate_final': False, 'b_net_amortized': True, 'experiment': 'nlg', 'sigma_x': 0.01, 'sigma_y': 0.01, 'beta': 1.0, 'num_dims': 1, 'num_sims': 10000, 'num_iters': 50000, 'non_linearity': 'exp', 'step_size': 0.01, 'log_dir': './out/2024-06-12/run_2024-06-12_12-40-34', 'dump_dir': 'exp/2024-06-12/run_2024-06-12_12-40-34', 'b_net_num_grad_steps': 2000, 'b_net_optimizer': 'adam-w', 'b_net_scheduler': 'none', 'b_net_lr': 0.001, 'random_seed': 128, 'device': device(type='cuda'), 'mc_config': {'num_samples': 100}}


100%|██████████| 49999/49999 [00:13<00:00, 3805.25it/s]


In [4]:
batch = experiment_nlg_exp.get_batch()
print("BEFORE STANDARDIZATION\n")
for k, v in batch.items():
    print(k, "-> mean: ", v.mean(), ", std: ", v.std())
batch = experiment_nlg_exp.standardize(batch)
print("\nAFTER STANDARDIZATION\n")
for k, v in batch.items():
    print(k, "-> mean: ", v.mean(), ", std: ", v.std())

BEFORE STANDARDIZATION

x0 -> mean:  tensor(5.4229) , std:  tensor(1.4124)
x1 -> mean:  tensor(5.4229) , std:  tensor(1.4123)
xc -> mean:  tensor(5.4229) , std:  tensor(1.4124)
y -> mean:  tensor(5.4229) , std:  tensor(1.4124)

AFTER STANDARDIZATION

x0 -> mean:  tensor(-2.6334e-05) , std:  tensor(1.2679)
x1 -> mean:  tensor(-3.2826e-05) , std:  tensor(1.2677)
xc -> mean:  tensor(-2.6334e-05) , std:  tensor(1.2679)
y -> mean:  tensor(-2.8455e-05) , std:  tensor(1.2679)


In [5]:
## initializing optimizer
b_net_optimizer = OPTIMIZERS[b_net_optimizer](experiment_nlg_exp.b_net.backbone.parameters(), lr = b_net_lr)

## constructing optimization config dictionary
b_net_optim_config = {
    "num_grad_steps": b_net_num_grad_step,
    "optimizer": b_net_optimizer,
    "scheduler": b_net_scheduler
}

## training b_net 
experiment_nlg_exp.train(b_net_optim_config)
## saving the weights
torch.save(experiment_nlg_exp.b_net.state_dict(), os.path.join(dump_dir, "b_net_exp.pt"))

Grad Step 2000/2000, MSELoss: 0.7678613662719727: 100%|██████████| 2000/2000 [06:19<00:00,  5.27it/s]
